## Oracle 
--- 
Forecast - Forms
1.  Export Jax forecast, save as JAX FC.csv in Resources folder.
2.  Export Lex forecast, save as LEX FC.csv in Resources folder.
----
Open Orders - All WABCUST Reports, WABCUST YTS OPEN ORDER REPORT - XML
3.  Org ID - Jax, Exclude order type:  YTS - Internal Orders, save as JAX OPEN ORDERS.csv in Resources folder.
4.  Org ID - Lex, Exclude order type:  YTS - Internal Orders, save as LEX OPEN ORDERS.csv in Resources folder.


In [1]:
# Import libraries
import pandas as pd
#from pandas import ExcelWriter
#from pandas import ExcelFile
#import datetime
#import matplotlib as plt
import numpy as np

## Jax forecast clean up

In [2]:
# Load jax fc file
jax_fc_file = "Resources/JAX FC.csv"

# Read csv file into DataFrame, set Item as index
#jax_fc = pd.read_csv(jax_fc_file, index_col = "Item", parse_dates=["Date"])
jax_fc = pd.read_csv(jax_fc_file, index_col = "Item")
jax_fc.head()

,Bucket,Date,End Date,Number of Buckets,Current,Original,Total,Confidence(%),Project Number,Unnamed: 10,Task Number,Line,Comment,Forecast,[ ]
Item,,,,,,,,,,,,,,,
YT363694-805,Days,12/29/2020,NaN,1,0,0,0,100,NaN,NaN,NaN,NaN,"PR 12,726.82",JAX-FCST,.
YT413234-805,Days,8/31/2020,NaN,1,0,0,0,100,NaN,NaN,NaN,NaN,PR 6229.00,JAX-FCST,.
YT425655,Days,10/27/2020,NaN,1,35,52,35,100,NaN,NaN,NaN,NaN,CAT 62.00,JAX-FCST,.
YT425663-803,Days,11/10/2020,NaN,1,8,8,8,100,NaN,NaN,NaN,NaN,CAT 7042.00,JAX-FCST,.
YT425663-815,Days,9/28/2020,NaN,1,0,0,0,100,NaN,NaN,NaN,NaN,CAT 6889.00,JAX-FCST,.


In [3]:
# Remove extra columns 
jax_fc = jax_fc[["Date", "Current", "Comment"]]
jax_fc.head()

,Date,Current,Comment
Item,,,
YT363694-805,12/29/2020,0,"PR 12,726.82"
YT413234-805,8/31/2020,0,PR 6229.00
YT425655,10/27/2020,35,CAT 62.00
YT425663-803,11/10/2020,8,CAT 7042.00
YT425663-815,9/28/2020,0,CAT 6889.00


In [4]:
# New data frame with split value columns for Comment column 
new = jax_fc["Comment"].str.split(" ", n = 1, expand = True) 
new.head()

,0,1
Item,,
YT363694-805,PR,"12,726.82"
YT413234-805,PR,6229.00
YT425655,CAT,62.00
YT425663-803,CAT,7042.00
YT425663-815,CAT,6889.00


In [5]:
# Make separate Customer column from new data frame 
new["Customer"]= new[0] 
new.head()

,0,1,Customer
Item,,,
YT363694-805,PR,"12,726.82",PR
YT413234-805,PR,6229.00,PR
YT425655,CAT,62.00,CAT
YT425663-803,CAT,7042.00,CAT
YT425663-815,CAT,6889.00,CAT


In [6]:
# Make separate Price column from new data frame 
new["Price"]= new[1] 
new.head()

,0,1,Customer,Price
Item,,,,
YT363694-805,PR,"12,726.82",PR,"12,726.82"
YT413234-805,PR,6229.00,PR,6229.00
YT425655,CAT,62.00,CAT,62.00
YT425663-803,CAT,7042.00,CAT,7042.00
YT425663-815,CAT,6889.00,CAT,6889.00


In [7]:
# Dropping 0 and 1 columns from new data frame
new.drop(columns =[0, 1], inplace = True)
new.head()

,Customer,Price
Item,,
YT363694-805,PR,"12,726.82"
YT413234-805,PR,6229.00
YT425655,CAT,62.00
YT425663-803,CAT,7042.00
YT425663-815,CAT,6889.00


In [8]:
# Merge jax_fc and new data frames
total_jax_fc = pd.merge(jax_fc, new, on="Item")
total_jax_fc

,Date,Current,Comment,Customer,Price
Item,,,,,
YT363694-805,12/29/2020,0,"PR 12,726.82",PR,"12,726.82"
YT413234-805,8/31/2020,0,PR 6229.00,PR,6229.00
YT425655,10/27/2020,35,CAT 62.00,CAT,62.00
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.00
YT425663-815,9/28/2020,0,CAT 6889.00,CAT,6889.00
...,...,...,...,...,...
YT50972-113.50,12/29/2020,0,PR 17.00,PR,17.00
YT50972-113.50,12/29/2020,0,PR 17.00,PR,17.00
YT50972-113.50,12/29/2020,0,PR 17.00,PR,17.00


In [9]:
# Change Current to Qty
total_jax_fc = total_jax_fc.rename(columns = {"Current": "Qty"})
total_jax_fc.head()

,Date,Qty,Comment,Customer,Price
Item,,,,,
YT363694-805,12/29/2020,0,"PR 12,726.82",PR,"12,726.82"
YT413234-805,8/31/2020,0,PR 6229.00,PR,6229.00
YT425655,10/27/2020,35,CAT 62.00,CAT,62.00
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.00
YT425663-815,9/28/2020,0,CAT 6889.00,CAT,6889.00


In [10]:
# Remove Qty 0
total_jax_fc = total_jax_fc[total_jax_fc["Qty"] != 0]
total_jax_fc.head()

,Date,Qty,Comment,Customer,Price
Item,,,,,
YT425655,10/27/2020,35,CAT 62.00,CAT,62.00
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.00
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.00
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.00
YT427896-805,11/19/2020,4,PR 9119.00,PR,9119.00


In [11]:
#Check data types
total_jax_fc.dtypes

Date        object
Qty          int64
Comment     object
Customer    object
Price       object
dtype: object

In [12]:
# Change Price data type from object to float
total_jax_fc['Price'] = total_jax_fc['Price'].astype(float)
total_jax_fc

,Date,Qty,Comment,Customer,Price
Item,,,,,
YT425655,10/27/2020,35,CAT 62.00,CAT,62.0
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.0
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0
YT427896-805,11/19/2020,4,PR 9119.00,PR,9119.0
...,...,...,...,...,...
YT442954-811,12/9/2020,2,CAT 13640.00,CAT,13640.0
YT442954-811,12/9/2020,2,CAT 13640.00,CAT,13640.0
YT442954-811,12/9/2020,2,CAT 13640.00,CAT,13640.0


In [13]:
# Recheck data types
total_jax_fc.dtypes

Date         object
Qty           int64
Comment      object
Customer     object
Price       float64
dtype: object

In [14]:
# Check total count against resource file to ensure data is correct
total_jax_fc.count()

Date        85
Qty         85
Comment     85
Customer    85
Price       85
dtype: int64

In [15]:
# Add extended price column and populate with Price * Qty
total_jax_fc["Extended Price"] = total_jax_fc.Price * total_jax_fc.Qty
total_jax_fc.head()

,Date,Qty,Comment,Customer,Price,Extended Price
Item,,,,,,
YT425655,10/27/2020,35,CAT 62.00,CAT,62.0,2170.0
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.0,56336.0
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0,21140.0
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0,21140.0
YT427896-805,11/19/2020,4,PR 9119.00,PR,9119.0,36476.0


In [16]:
# Add column TYPE
total_jax_fc["Type"] = "FC"
total_jax_fc.head()

,Date,Qty,Comment,Customer,Price,Extended Price,Type
Item,,,,,,,
YT425655,10/27/2020,35,CAT 62.00,CAT,62.0,2170.0,FC
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.0,56336.0,FC
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0,21140.0,FC
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0,21140.0,FC
YT427896-805,11/19/2020,4,PR 9119.00,PR,9119.0,36476.0,FC


In [17]:
# Add column Org
total_jax_fc["Org"] = "JAX"
total_jax_fc.head()

,Date,Qty,Comment,Customer,Price,Extended Price,Type,Org
Item,,,,,,,,
YT425655,10/27/2020,35,CAT 62.00,CAT,62.0,2170.0,FC,JAX
YT425663-803,11/10/2020,8,CAT 7042.00,CAT,7042.0,56336.0,FC,JAX
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0,21140.0,FC,JAX
YT426744-AB-803,11/17/2020,2,CAT 10570.00,CAT,10570.0,21140.0,FC,JAX
YT427896-805,11/19/2020,4,PR 9119.00,PR,9119.0,36476.0,FC,JAX


In [18]:
# New data frame with deleted Comment column
final_jax_fc = total_jax_fc.drop(["Comment"], axis = 1)
final_jax_fc

,Date,Qty,Customer,Price,Extended Price,Type,Org
Item,,,,,,,
YT425655,10/27/2020,35,CAT,62.0,2170.0,FC,JAX
YT425663-803,11/10/2020,8,CAT,7042.0,56336.0,FC,JAX
YT426744-AB-803,11/17/2020,2,CAT,10570.0,21140.0,FC,JAX
YT426744-AB-803,11/17/2020,2,CAT,10570.0,21140.0,FC,JAX
YT427896-805,11/19/2020,4,PR,9119.0,36476.0,FC,JAX
...,...,...,...,...,...,...,...
YT442954-811,12/9/2020,2,CAT,13640.0,27280.0,FC,JAX
YT442954-811,12/9/2020,2,CAT,13640.0,27280.0,FC,JAX
YT442954-811,12/9/2020,2,CAT,13640.0,27280.0,FC,JAX


In [19]:
# Check total count against resource file to ensure data is correct
final_jax_fc.count()

Date              85
Qty               85
Customer          85
Price             85
Extended Price    85
Type              85
Org               85
dtype: int64

In [20]:
# Sort by Date, then Item 
final_jax_fc = final_jax_fc.sort_values(["Item", "Date"])
final_jax_fc.head()

,Date,Qty,Customer,Price,Extended Price,Type,Org
Item,,,,,,,
YT425655,10/27/2020,35,CAT,62.0,2170.0,FC,JAX
YT425663-803,11/10/2020,8,CAT,7042.0,56336.0,FC,JAX
YT426744-AB-803,11/17/2020,2,CAT,10570.0,21140.0,FC,JAX
YT426744-AB-803,11/17/2020,2,CAT,10570.0,21140.0,FC,JAX
YT427896-805,1/18/2021,3,PR,9119.0,27357.0,FC,JAX


In [21]:
# Reorder columns
#final_jax_fc = final_jax_fc[["Date", "Qty", "Org", "Type"]]
#final_jax_fc.head()

In [22]:
# Save file to csv
final_jax_fc.to_csv("output_data/final_jax_fc.csv")

In [23]:
# Save to Excel
final_jax_fc.to_excel("output_data/final_jax_fc.xlsx")

## Lex forecast clean up

In [24]:
# Load lex fc file
lex_fc_file = "Resources/LEX FC.csv"

# Read csv file into DataFrame, set Item as index
#lex_fc = pd.read_csv(lex_fc_file, index_col = "Item", parse_dates=["Date"])
lex_fc = pd.read_csv(lex_fc_file, index_col = "Item")
lex_fc.head()

,Bucket,Date,End Date,Number of Buckets,Current,Original,Total,Confidence(%),Project Number,Unnamed: 10,Task Number,Line,Comment,Forecast,[ ]
Item,,,,,,,,,,,,,,,
YT05627-144,Days,10/9/2020,NaN,1,2,4,2,100,NaN,NaN,NaN,NaN,CAT 301.00,LEX-FCST,.
YT05627-144,Days,10/20/2020,NaN,1,4,4,4,100,NaN,NaN,NaN,NaN,CAT 301.00,LEX-FCST,.
YT05627-144,Days,11/5/2020,NaN,1,2,2,2,100,NaN,NaN,NaN,NaN,CAT 301.00,LEX-FCST,.
YT05627-144,Days,11/18/2020,NaN,1,3,3,3,100,NaN,NaN,NaN,NaN,CAT 301.00,LEX-FCST,.
YT05627-144,Days,12/7/2020,NaN,1,4,4,4,100,NaN,NaN,NaN,NaN,CAT 301.00,LEX-FCST,.


In [25]:
# Remove extra columns 
lex_fc = lex_fc[["Date", "Current", "Comment"]]
lex_fc.head()

,Date,Current,Comment
Item,,,
YT05627-144,10/9/2020,2,CAT 301.00
YT05627-144,10/20/2020,4,CAT 301.00
YT05627-144,11/5/2020,2,CAT 301.00
YT05627-144,11/18/2020,3,CAT 301.00
YT05627-144,12/7/2020,4,CAT 301.00


In [26]:
# New data frame with split value columns for Comment column 
new2 = lex_fc["Comment"].str.split(" ", n = 1, expand = True) 
new2.head()

,0,1
Item,,
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00


In [27]:
# Make separate Customer column from new data frame 
new2["Customer"]= new2[0] 
new2

,0,1,Customer
Item,,,
YT05627-144,CAT,301.00,CAT
YT05627-144,CAT,301.00,CAT
YT05627-144,CAT,301.00,CAT
YT05627-144,CAT,301.00,CAT
YT05627-144,CAT,301.00,CAT
...,...,...,...
YT446936,11504,None,11504
YT446936,11504,None,11504
YT446936,11504,None,11504


In [28]:
# Make separate Price column from new data frame 
new2["Price"]= new2[1] 
new2

,0,1,Customer,Price
Item,,,,
YT05627-144,CAT,301.00,CAT,301.00
YT05627-144,CAT,301.00,CAT,301.00
YT05627-144,CAT,301.00,CAT,301.00
YT05627-144,CAT,301.00,CAT,301.00
YT05627-144,CAT,301.00,CAT,301.00
...,...,...,...,...
YT446936,11504,None,11504,None
YT446936,11504,None,11504,None
YT446936,11504,None,11504,None


In [29]:
# Dropping 0 and 1 columns from new data frame
new2.drop(columns =[0, 1], inplace = True)
new2

,Customer,Price
Item,,
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
YT05627-144,CAT,301.00
...,...,...
YT446936,11504,None
YT446936,11504,None
YT446936,11504,None


## Issue starts here...

In [30]:
# Merge lex_fc and new data frames
total_lex_fc = pd.merge(lex_fc, new2, on="Item")
total_lex_fc

,Date,Current,Comment,Customer,Price
Item,,,,,
YT05627-144,10/9/2020,2,CAT 301.00,CAT,301.00
YT05627-144,10/9/2020,2,CAT 301.00,CAT,301.00
YT05627-144,10/9/2020,2,CAT 301.00,CAT,301.00
YT05627-144,10/9/2020,2,CAT 301.00,CAT,301.00
YT05627-144,10/9/2020,2,CAT 301.00,CAT,301.00
...,...,...,...,...,...
YT446936,9/17/2021,1,11504,11504,None
YT446936,9/17/2021,1,11504,11504,None
YT446936,9/17/2021,1,11504,11504,None


In [ ]:
# Change Current to Qty
total_lex_fc = total_lex_fc.rename(columns = {"Current": "Qty"})
total_lex_fc.head()

In [ ]:
# Remove Qty 0
total_lex_fc = total_lex_fc[total_lex_fc["Qty"] != 0]
total_lex_fc.head()

In [ ]:
# Check data types
total_lex_fc.dtypes

In [ ]:
# Change Price data type from object to float
#total_lex_fc['Price'] = total_lex_fc['Price'].astype(float)
#total_lex_fc

In [ ]:
# Recheck data types
total_lex_fc.dtypes

In [ ]:
# Check total count against resource file to ensure data is correct
total_lex_fc.count()

In [ ]:
# Rename Current to Open Qty and Date to Schedule Date to match open orders
lex_fc = lex_fc.rename(columns = {"Current": "Qty"})
lex_fc.head()

In [ ]:
# Add columns Customer, PC, Price, Extended Price, Org, Type
# Add column ORG, TYPE
#lex_fc["Customer"] = None
#lex_fc["PC"] = None
#lex_fc["Price"] = None
#lex_fc["Extended Price"] = None
#lex_fc["Org"] = "LEX"

# Add column TYPE
lex_fc["Type"] = "FC"
lex_fc.head()

In [ ]:
# Reorder columns
#lex_fc = lex_fc[["Date", "Qty", "Customer", "PC", "Price", "Extended Price", "Org", "Type"]]
#lex_fc.head()

In [ ]:
# Remove Open Qty 0
lex_fc = lex_fc[lex_fc["Qty"] != 0]
lex_fc.head()

In [ ]:
# Check data types
lex_fc.dtypes

In [ ]:
# Filter dates before 12/31/2020
#updated_lex_fc= lex_fc.loc[lex_fc["Date"] <= "12-31-2020"]
#updated_lex_fc

In [ ]:
# Filter dates to end on 12/31/2020
#end_date = "12/31/2020"
#lex_fc = (lex_fc['Date']>= end_date )
#lex_fc

In [ ]:
#Sort by Schedule Date, then Item 
lex_fc = lex_fc.sort_values(["Item", "Date"])
lex_fc

In [ ]:
#Check total count against resource file to ensure data is correct
lex_fc.count()

In [ ]:
# Save file to csv
lex_fc.to_csv("output_data/lex_fc.csv")

In [ ]:
# Save to Excel
#lex_fc.to_excel("output_data/lex_fc.xlsx")

## Merge Jax forecast and Lex forecast

In [ ]:
# Merge jax_fc and lex_fc
both = [jax_fc, lex_fc]
total_fc = pd.concat(both)
total_fc.head()

In [ ]:
# Check count against jax plus lex fc counts
total_fc.count()

In [ ]:
# Save file to csv
total_fc.to_csv("output_data/total_fc.csv")

## Merge total_fc file with LOOKUP file

In [ ]:
# Load lookup file
lookup_file = "Resources/LOOKUP.xlsx"

# Read excel file into DataFrame, set Item as index
#jax_fc = pd.read_csv(jax_fc_file, index_col = "Item", parse_dates=["Date"])
lookup = pd.read_excel(lookup_file, index_col = "Item")
lookup.head()

In [ ]:
#Reorder columns
lookup = lookup[["Customer", "PC", "Price", "Org"]]
lookup.head()

In [ ]:
# Merge total_fc and lookup files
complete_fc = pd.merge(total_fc, lookup, on = "Item")
complete_fc

# Important note:
# Check total number of rows on complete_fc should equal to jax_fc plus lex_fc

In [ ]:
# Add extended price column and populate with Price * Qty
complete_fc["Extended Price"] = complete_fc.Price * complete_fc.Qty
complete_fc

In [ ]:
# Reorder columns to match sales order columns
complete_fc = complete_fc[["Date", "Qty", "Customer", "PC", "Price", "Extended Price", "Org", "Type"]]
complete_fc

In [ ]:
# Check row count again
complete_fc.count()

In [ ]:
# Save file to csv
complete_fc.to_csv("output_data/complete_fc.csv")

## Jax open orders clean up

In [ ]:
# Load jax so file
jax_so_file = "Resources/JAX OPEN ORDERS.csv"

# Read csv file into DataFrame, set Item as index
jax_so = pd.read_csv(jax_so_file, index_col = "Item", encoding = "ISO-8859–1")
jax_so.head()

In [ ]:
# List of columns
list(jax_so.columns)

In [ ]:
# Narrow down columns 
jax_so = jax_so[["Schedule Date", "Open Qty", "Customer Name", "Planner", "Price", "Extended Price"]]
jax_so.head()

In [ ]:
# Rename Open Qty to Qty and Date to Schedule Date to match open orders
jax_so = jax_so.rename(columns = { "Schedule Date" : "Date",
                                  "Open Qty": "Qty", 
                                  "Customer Name": "Customer",
                                  "Planner": "PC",
                                 })
jax_so.head()

In [ ]:
# Add column ORG, TYPE
jax_so["Org"] = "JAX"
jax_so["Type"] = "SO"
jax_so.head()

In [ ]:
# Remove buy parts by planner code
#df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]
#jax_so.loc[(jax_so["PC"] == "3")]

In [ ]:
# Check data types
jax_so.dtypes

In [ ]:
# Check count
jax_so.count()

In [ ]:
# Save file to csv
jax_so.to_csv("output_data/jax_so.csv", date_format='%Y-%m-%d')

In [ ]:
# Save to Excel
#jax_so.to_excel("output_data/jax_so.xlsx")

## Lex open orders clean up

In [ ]:
# Load lex so file
lex_so_file = "Resources/LEX OPEN ORDERS.csv"

# Read csv file into DataFrame, set Item as index
lex_so = pd.read_csv(lex_so_file, index_col = "Item", encoding = "ISO-8859–1")
lex_so.head()

In [ ]:
# List of columns
list(lex_so.columns)

In [ ]:
# Narrow down columns 
lex_so = lex_so[["Schedule Date", "Open Qty", "Customer Name", "Planner", "Price", "Extended Price"]]
lex_so.head()

In [ ]:
# Rename Open Qty to Qty and Date to Schedule Date to match open orders
lex_so = lex_so.rename(columns = { "Schedule Date" : "Date",
                                  "Open Qty": "Qty", 
                                  "Customer Name": "Customer",
                                  "Planner": "PC",
                                 })
lex_so.head()

In [ ]:
# Add column ORG, TYPE
lex_so["Org"] = "LEX"
lex_so["Type"] = "SO"
lex_so.head()

In [ ]:
# Remove buy parts by planner code
#lex_so.loc[(lex_so["PC"] == "5")]

In [ ]:
lex_so.dtypes

In [ ]:
# Check count
lex_so.count()

In [ ]:
# Save file to csv
lex_so.to_csv("output_data/lex_so.csv", date_format='%Y-%m-%d')

In [ ]:
# Save to Excel
#lex_so.to_excel("output_data/lex_so.xlsx")

## Merge Jax open orders and Lex open orders

In [ ]:
# Merge jax_so and lex_so
# Use pd.concat to add one file to the other instead of actual merging
both = [jax_so, lex_so]
total_so = pd.concat(both)
total_so.head()

In [ ]:
# Check count
total_so.count()

In [ ]:
# Save file to csv
total_so.to_csv("output_data/total_so.csv")

## Merge complete_fc file with total_so file

In [ ]:
# Merge complete_fc and total_so files
#complete_fc_so = pd.merge(complete_fc, total_so, on = "Item")
#complete_fc_so

# Important note:
# Check total number of rows on complete_fc should equal to jax_fc plus lex_fc

In [ ]:
# Merge complete_fc and total_so files
# Use pd.concat to add one file to the other instead of actual merging
both_files = [complete_fc, total_so]
complete_fc_so = pd.concat(both)
complete_fc_so

In [ ]:
# Save file to csv
complete_fc_so.to_csv("output_data/complete_fc_so.csv")

## Merge complete_fc_so file with LOOKUP file
## Add eq units to complete_fc_so file